# Author Identification
#### Carl Moser, Matthew Beaudouin-Lafon

We did stuff to do things with doodas.

In [43]:
from thinkbayes2 import Suite
from pickle import load
import thinkplot

def getPickle(fileName):
    infile = open(fileName, 'rb+')
    chain = load(infile)
    wordCount = load(infile)
    author = load(infile)
    infile.close()
    return (chain, wordCount, author)

def isWorthChecking(prevWord, word):
    worth = True
    for author, prob in authorId.Items():
        chain = authorId.markovChains[author][0]
        worth = worth and (bool(chain.get(prevWord)) and bool(chain[prevWord].get(word)))# and (chain[prevWord][word]/float(wordCount[prevWord]) > 1e-4)
#     if not worth and word == 'unpack':
#         print(prevWord, word)
    if worth == None:
        print (bool(chain.get(prevWord)) and bool(chain[prevWord].get(word)))
    return worth

class AuthorId(Suite):
    """
    
    """
                
    def Likelihood(self, data, hypo):
        """
        data: (string, string)
        hypo: (Markov Chain, Word Count dictionary, Author)
        """
        hypoAuthor = hypo
        chain, wordCount = self.markovChains[hypoAuthor]
        prevWord, word = data
        if (chain.get(prevWord) and chain[prevWord].get(word)):
            like = chain[prevWord][word]/float(wordCount[prevWord])
        else:      
            like = 0.000001 # It's 0.01 because 0.01 is a number
#         print(hypoAuthor, '      ', like)
        return like

In [2]:
# files = ['Frankenstein', 'GreatExpecations', 'Hamlet', 'MobyDick', 'PierreAmbiguities', 'RomeoAndJuliet']
files = ['Frankenstein.dat', 'GreatExp.dat', 'RomeoJuliet.dat']

authorId = AuthorId()
authorId.markovChains = {}

for f in files:
    chain, wordCount, author = getPickle(f)
    authorId[author] = 1
    authorId.markovChains[author] = (chain, wordCount)

In [3]:
f = open('testText.dat')
testText = load(f)
f.close()

In [44]:
for author, prob in authorId.Items():
    authorId[author] = 1
    
authorId.Normalize()
maryV = []
charlesV = []
shakeV = []
totalWords = 0
for sentence in testText:
    sentence = ['*'] + sentence
#     print(sentence)
    for i in range(len(sentence) - 1):
        totalWords += 1
        if totalWords == 233:
            print((sentence[i], sentence[i + 1]), isWorthChecking(sentence[i], sentence[i + 1]))
        if isWorthChecking(sentence[i], sentence[i + 1]):
            authorId.Update((sentence[i], sentence[i + 1]))
            
        maryV.append(authorId['Mary Shelley'])
        charlesV.append(authorId['Charles Dickens'])
        shakeV.append(authorId['Shakespeare'])

(('alone', 'to'), False)


In [49]:
import matplotlib.pyplot as pyplot

for i in (shakeV):
    print(str(i) + ', ')

# for fuck, shit in zip(maryV, charlesV):
#     print(fuck, shit)
# authorId

0.333333333333, 
0.333333333333, 
0.333333333333, 
0.710270564103, 
0.710270564103, 
0.710270564103, 
0.453082827345, 
0.611795162391, 
0.611795162391, 
0.611795162391, 
0.611795162391, 
0.611795162391, 
0.447262044587, 
0.447262044587, 
0.447262044587, 
0.447262044587, 
0.447262044587, 
0.447262044587, 
0.11780546176, 
0.11780546176, 
0.11780546176, 
0.11780546176, 
0.11780546176, 
0.118296104797, 
0.118296104797, 
0.118296104797, 
0.118296104797, 
0.118296104797, 
0.118296104797, 
0.118296104797, 
0.118296104797, 
0.102104671437, 
0.102104671437, 
0.102104671437, 
0.102104671437, 
0.102104671437, 
0.102104671437, 
0.102104671437, 
0.102104671437, 
0.102104671437, 
0.0877386276106, 
0.0877386276106, 
0.0877386276106, 
0.0877386276106, 
0.0877386276106, 
0.0877386276106, 
0.0877386276106, 
0.0391310948861, 
0.0391310948861, 
0.0391310948861, 
0.0391310948861, 
0.0391310948861, 
0.0391310948861, 
0.0124617615605, 
0.0105220717895, 
0.0105220717895, 
0.00501002929832, 
0.00501002929832, 

In [23]:
chainC = authorId.markovChains['Charles Dickens'][0]
countC = authorId.markovChains['Charles Dickens'][1]

chainM = authorId.markovChains['Mary Shelley'][0]
countM = authorId.markovChains['Mary Shelley'][1]

chainS = authorId.markovChains['Shakespeare'][0]
countS = authorId.markovChains['Shakespeare'][1]

print(chainM['unpack']['and'])

KeyError: 'unpack'

Questions:
    - What to do with words that aren't there?
    - "To" appears a lot more than other words. Should it be weighed differently?
    - Integrate sentence lengths